In [1]:
sc

In [3]:
dfTweets = sqlContext.read.load('samplestream_twt.jsonl', format='json')
# dfTweets.printSchema()

In [56]:
retweet_status = dfTweets.select('place.bounding_box', 'place.full_name', 'timestamp_ms')
retweet_status.registerTempTable('toplocations')
occurrence = sqlContext.sql('select bounding_box, full_name, timestamp_ms as occurrence from toplocations').dropna() 
# toploc = occurrence.orderBy('occurrence', ascending=False) #order by descending order based on occurrence
occurrence.show()

+--------------------+----------------+-------------+
|        bounding_box|       full_name|   occurrence|
+--------------------+----------------+-------------+
|[WrappedArray(Wra...|   Millville, NJ|1494455868620|
|[WrappedArray(Wra...|   Arkansas, USA|1494455868465|
|[WrappedArray(Wra...|   Pittsburg, KS|1494455870393|
|[WrappedArray(Wra...|      Downey, CA|1494455871012|
|[WrappedArray(Wra...|      Austin, TX|1494455870905|
|[WrappedArray(Wra...|Indianapolis, IN|1494455872670|
|[WrappedArray(Wra...|        Lodi, NJ|1494455871877|
|[WrappedArray(Wra...| Mercerville, NJ|1494455874700|
|[WrappedArray(Wra...|   New York, USA|1494455874732|
|[WrappedArray(Wra...|    Florida, USA|1494455874850|
|[WrappedArray(Wra...|      Texas, USA|1494455876561|
|[WrappedArray(Wra...|   Manhattan, NY|1494455876675|
|[WrappedArray(Wra...|   Pittsburg, CA|1494455877895|
|[WrappedArray(Wra...|   San Diego, CA|1494455878895|
|[WrappedArray(Wra...|Rimouski, Québec|1494455879774|
|[WrappedArray(Wra...|    Al

In [64]:
# This command is what we use in the cluster.
occurrence.rdd.saveAsTextFile('finaltoplocationsout')

## After running the code above on cluster

In [65]:
toplocations = sc.textFile("finaltoplocationsout")

In [66]:
toplocations.take(1)

[u"Row(bounding_box=Row(coordinates=[[[-75.101446, 39.342878], [-75.101446, 39.429603], [-74.989836, 39.429603], [-74.989836, 39.342878]]], type=u'Polygon'), full_name=u'Millville, NJ', occurrence=u'1494455868620')"]

In [67]:
remove = toplocations.map(lambda x: x.replace("Row(bounding_box=Row(coordinates=[[[", ""))
remove = remove.map(lambda x: x.replace("]], type=u'Polygon'), full_name=u'", ","))
remove = remove.map(lambda x: x.replace(", occurrence=u", ";"))
final = remove.map(lambda x: x.replace(")", "").replace("],", ';').replace('[', '').replace(']', '').replace("'", ''))

final.take(5)

[u'-75.101446, 39.342878; -75.101446, 39.429603; -74.989836, 39.429603; -74.989836, 39.342878;Millville, NJ;1494455868620',
 u'-94.61771, 33.004106; -94.61771, 36.499767; -89.644838, 36.499767; -89.644838, 33.004106;Arkansas, USA;1494455868465',
 u'-94.724428, 37.367357; -94.724428, 37.444178; -94.668019, 37.444178; -94.668019, 37.367357;Pittsburg, KS;1494455870393',
 u'-118.170429, 33.90245; -118.170429, 33.972897; -118.090423, 33.972897; -118.090423, 33.90245;Downey, CA;1494455871012',
 u'-97.928935, 30.127892; -97.928935, 30.518799; -97.580513, 30.518799; -97.580513, 30.127892;Austin, TX;1494455870905']

In [68]:
def mapper(k1v1s): 
    for k1v1 in k1v1s:
        k = k1v1.split(';')
        x_co = float(k[0].split(',')[0]) + float(k[1].split(',')[0])+ float(k[2].split(',')[0])+float(k[3].split(',')[0])
        x_co = x_co/4
        y_co = float(k[0].split(',')[1]) + float(k[1].split(',')[1])+ float(k[2].split(',')[1])+float(k[3].split(',')[1])
        y_co = y_co/4
        yield (round(y_co,6), round(x_co, 6), k[4], str(k[5]))

coordinates = final.mapPartitions(mapper)
coordinates.take(5)

[(39.38624, -75.045641, u'Millville, NJ', '1494455868620'),
 (34.751936, -92.131274, u'Arkansas, USA', '1494455868465'),
 (37.405767, -94.696224, u'Pittsburg, KS', '1494455870393'),
 (33.937674, -118.130426, u'Downey, CA', '1494455871012'),
 (30.323346, -97.754724, u'Austin, TX', '1494455870905')]

In [69]:
import csv

def writeRecords(records):
    with open('coordinates.csv', 'w') as csvfile:
        field = ['latitude', 'longitude', 'location','count']
        writer = csv.DictWriter(csvfile, fieldnames=field)

        writer.writeheader()
        for record in records:
            writer.writerow({'latitude': record[0], 'longitude': record[1], 'location': record[2], 'count': record[3]})

In [70]:
coo = coordinates.toLocalIterator()
writeRecords(coo)